In [1]:
import pandas as pd
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
from scripts.Processed import Processed

In [2]:
mvps = pd.read_csv('../data/raw/1981_2024_mvps.csv')
standings = pd.read_csv('../data/raw/1981_2024_standings.csv')
advanced = pd.read_csv('../data/raw/1981_2024_advanced.csv')
per_game = pd.read_csv('../data/raw/1981_2024_per_game.csv')
pro = Processed(mvps, standings, advanced, per_game)
print(standings.head(5))

   Year                Team     W     L   W/L%   GB   PS/G   PA/G   SRS  seed
0  1981      Boston Celtics  62.0  20.0  0.756    —  109.9  104.0  6.05     1
1  1981  Philadelphia 76ers  62.0  20.0  0.756    —  111.7  103.8  7.76     2
2  1981     Milwaukee Bucks  60.0  22.0  0.732    —  113.1  105.9  7.14     3
3  1981        Phoenix Suns  57.0  25.0  0.695    —  110.0  104.5  4.83     4
4  1981  Los Angeles Lakers  54.0  28.0  0.659  3.0  111.2  107.3  3.27     5


There is Divisions values inside standings so we want to remove that

In [3]:
standings = pro.remove_divisions()
print(standings.head(5))

   Year                Team     W     L   W/L%   GB   PS/G   PA/G   SRS  seed
0  1981      Boston Celtics  62.0  20.0  0.756    —  109.9  104.0  6.05     1
1  1981  Philadelphia 76ers  62.0  20.0  0.756    —  111.7  103.8  7.76     2
2  1981     Milwaukee Bucks  60.0  22.0  0.732    —  113.1  105.9  7.14     3
3  1981        Phoenix Suns  57.0  25.0  0.695    —  110.0  104.5  4.83     4
4  1981  Los Angeles Lakers  54.0  28.0  0.659  3.0  111.2  107.3  3.27     5


We only want Win percentage and we only want to get the abbreviation of each teams to correlate what we have with our MVPs datas

In [4]:
standings = pro.clean_stats_standings()
print(standings.head(5))

   Year                Team   W/L%  seed
0  1981      Boston Celtics  0.756     1
1  1981  Philadelphia 76ers  0.756     2
2  1981     Milwaukee Bucks  0.732     3
3  1981        Phoenix Suns  0.695     4
4  1981  Los Angeles Lakers  0.659     5


In [5]:
standings = pro.update_team_names()
print(standings.head(5))

   Year Team   W/L%  seed
0  1981  BOS  0.756     1
1  1981  PHI  0.756     2
2  1981  MIL  0.732     3
3  1981  PHO  0.695     4
4  1981  LAL  0.659     5


For each player in the MVPs datas we add the W/L% for their corresponding season and remove all non important or redondant columns

In [6]:
mvps = pro.clean_mvps()
print(mvps.head(5))

   Year               Player  Share
0  1981        Julius Erving  0.658
1  1981           Larry Bird  0.613
2  1981  Kareem Abdul-Jabbar  0.414
3  1981         Moses Malone  0.261
4  1981        George Gervin  0.120


In [7]:
mvps = pro.merge_stats()

We will need the advanced stats as well so we clean that df too and merge it with the mvps

In [8]:
mvps = pro.merge_stats_mvps()
print(mvps.head(5))

   Year                  Player   Age Team     G   PER    TS%   3PAr    FTr  \
0  1981          Adrian Dantley  25.0  UTA  80.0  24.3  0.622  0.004  0.482   
1  1981            Moses Malone  25.0  HOU  80.0  25.1  0.585  0.002  0.520   
2  1981              Larry Bird  24.0  BOS  82.0  19.9  0.528  0.049  0.218   
3  1981           Mike Mitchell  25.0  CLE  82.0  17.5  0.513  0.005  0.215   
4  1981  Michael Ray Richardson  25.0  NYK  79.0  17.1  0.511  0.091  0.303   

   ORB%  ...  ORB  DRB   TRB  AST  STL  BLK  TOV   PF   PTS  Share  
0   6.5  ...  2.4  4.0   6.4  4.0  1.4  0.2  3.5  3.1  30.7  0.022  
1  16.1  ...  5.9  8.8  14.8  1.8  1.0  1.9  3.9  2.8  27.8  0.261  
2   7.0  ...  2.3  8.6  10.9  5.5  2.0  0.8  3.5  2.9  21.2  0.613  
3   7.1  ...  2.6  3.5   6.1  1.7  0.8  0.6  2.1  2.4  24.5  0.004  
4   6.0  ...  2.2  4.7   6.9  7.9  2.9  0.4  3.8  3.3  16.4  0.010  

[5 rows x 50 columns]


Now we want to scale our data to have so we use StandardScaler

In [9]:
mvps_scaled = pro.scale()
print(mvps_scaled.head(5))
mvps_scaled.to_csv("../data/processed/mvps.csv", index=False)

   index       Age         G       PER       TS%      3PAr       FTr  \
0      2 -0.629022  1.134936  1.157215  0.148045 -0.721317 -0.410129   
1      4 -0.385716  1.016518  0.701123 -0.027207 -0.524885 -0.013106   
2      6  0.587508  1.134936  0.342765  0.178972 -0.950487  0.014919   
3      7  0.100896  1.095464  0.847724  0.436695 -0.908395 -0.382103   
4      8  1.560732  1.055991  2.069400  1.055232 -0.945810  0.341878   

       ORB%      DRB%      TRB%  ...       AST       STL       BLK       TOV  \
0  0.209367  1.486911  1.140893  ...  1.905645  2.828863  0.760745  2.728393   
1  0.003405 -0.030189 -0.048516  ...  3.183564  4.742301 -0.040373  3.091212   
2  0.394733  0.919914  0.818342  ...  0.308247  0.277614  0.159907  2.123695   
3 -0.078980 -0.658483 -0.451706  ...  0.521233  1.765843 -0.040373  1.639937   
4  0.332945  1.134453  0.999777  ...  0.787466  0.065010  4.966613  2.244635   

         PF       PTS      Share                  Player  Year  Team  
0  1.153769  2.

c:\Users\Samuel\miniconda3\envs\MVP_PREDICTOR\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\Users\Samuel\miniconda3\envs\MVP_PREDICTOR\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
